# Neural Networks
Neural networks can be constructed using the `torch.nn` package.

`nn` depends on `autograd` to define models and differentiate them. An `nn.Module` contains layers, and a method `forward(input)` that returns the `output`.

For example, below is a look at a letter classifier:
![](https://pytorch.org/tutorials/_images/mnist.png)

It is a simple feed-forward network, taking in an input, feeds it through several layers one after the other, and finally produce an output.

A typical training procedure for a neural network is as follows:
- Define the neural network that has some learnable parameters (or weights)
- Iterate overa dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back intop the network's parameters
- Update the weights of the network, typically using a simple update rulew: `weight = weight - learning_rate * gradient`

## Define the Network:
Let's define this netrwork:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 1 input channel, 6 output features, 5x5 kernels
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x,1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print (net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


With this, a `forward` function is defined, and `backward` can be called to compute the gradients through `autograd`. We can use any of the Tensor operations in the `forward` function.

The learnable parameters of a model are returned by `net.parameters()`

In [2]:
params = list(net.parameters())
print (len(params))
print (params[0].size()) # weights of conv1

10
torch.Size([6, 1, 5, 5])


Let's try a random 32x32 input. Note: expected input size of this net(LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print (out)

tensor([[-0.0357,  0.0771, -0.0261, -0.0875,  0.0452,  0.2251, -0.0068, -0.0493,
         -0.0207, -0.0633]], grad_fn=<AddmmBackward0>)


Zero the gradient buffers of all parameters and backpros with random gradients:

In [4]:
net.zero_grad()
out.backward(torch.randn(1,10))

Note: `torch.nn` only supports mini-batches. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, `nn.Conv2d` will take in a 4D Tensor of `nSamples x nChannels x Height x Width`.

If we have a single sample, just use `input.unsqueeze(0)` to add an empty dimension of axis 0.

### Recap:
- `torch.Tensor` - A _multi-dimensional array_ with support for autograd operations like `backward()`. Also _holds gradients_ w.r.t. the tensor.
- `nn.Module` - Neural Network module. _Convevient way of encapsulating parameters_, with helpers for moving them to GPU, exporting, loading, etc.
- `nn.Parameters` - A Tensor _automatically registered as a parameter when assigned as an attribute_ to a `Module`.
- `autograd.Function` - Implements _forward_ and _backward_ definitions of an _autograd operation_. Every `Tensor` operation creates at least a single `Function` node that connects to functions that created a `Tensor` and _encodes its history_.
- We also defined a neural network class with `nn.Module`
- Demonstrated processing inputs and the `.backward()` function

## Loss Function
A loss function is a measure of how far away the output is from the target. There are several [loss functions](https://pytorch.org/docs/nn.html#loss-functions) under the `nn` package. A commonly used loss function is `nn.MSELoss` which computes the mean-squared error between the output and the target.

In [5]:
output = net(input)
target = torch.randn(10) # dummy target
target = target.view(1, -1) # make it same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print (loss)

tensor(1.0993, grad_fn=<MseLossBackward0>)


If we follow `loss` in the backward direction, using its `.grad_fn` attribute, we see a graph of comutations that look like:

`input  -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d 
        -> flatten -> linear -> relu -> linear -> relu -> linear
        -> MSELoss
        -> loss
`

So when we call `loss.backward()` the whole graph is differentiated w.r.t. the neural network parameters, and all Tensors in the graph that have `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient.

For illustration, let's follow a few steps backward:

In [6]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop
To backpropagate the error, all we have to do is call `loss.backward()`. We need to clear the existing gradients, else new gradients will be accumulated onto existing gradients.

Now we call `loss.backward()`, and have a look at conv1's bias gradients before and fater the backward.

In [7]:
net.zero_grad()

print ('conv1.bias.grad before backward')
print (net.conv1.bias.grad)

loss.backward()
print ('conv1.bias.grad before backward')
print (net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad before backward
tensor([ 0.0068, -0.0036,  0.0027, -0.0134,  0.0136, -0.0031])


## Update the Weights
With the gradients propagated backwrds, we can now use them to update the weights of our network. A common optimization method is stochastic gradient descent (SGD):

`weight = weight - learning_rate * gradient`

We can implement this using Python code:

In [8]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, PyTorch provides a more flexible library that allows us to change optimization methods from SGD to Adam, RMSPros, etc. with the `torch.optim` package.

In [9]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 1e-2)

optimizer.zero_grad()
output = net(input)
loss = criterion (output, target)
loss.backward()
optimizer.step()

Note: Observe how the buffers had to be manually set to zero using `optimizer.zero_grad()`. This is bacause gradients are accumulated by default as explained previously.